In [ ]:
"""
author: Miranda Lv
Date: Aug, 2023
"""

In [4]:
import rasterio
import os
from os.path import dirname as up
import glob
from pathlib import Path

import geopandas as gpd
import fiona
from shapely.geometry import box, Polygon
import numpy as np

import matplotlib.pyplot as plt
from rasterio.plot import show

In [2]:
root_path = up(os.path.abspath('.'))
tiffDir = os.path.join(root_path, 'data/output')
aoi = os.path.join(root_path, 'data/processing_data/vectors/subregion_aoi_wgs84.geojson')

In [5]:
alltif = list(Path(tiffDir).glob("*.tif"))

In [6]:
def get_poly_bound(gdf):
    
    bounds = gdf['geometry'].bounds
    minx = np.min(bounds.minx)
    miny = np.min(bounds.miny)
    maxx = np.max(bounds.maxx)
    maxy = np.max(bounds.maxy)
    
    return minx, miny, maxx, maxy


In [15]:
aio_gdf = gpd.read_file(aoi)
aio_gdf = aio_gdf.to_crs(32618)
bounds = aio_gdf['geometry'].bounds

minx, miny, maxx, maxy = get_poly_bound(aio_gdf)
new_poly = box(minx, miny, maxx, maxy)


In [19]:
poly = [{'type': 'Polygon',
  'coordinates': [[new_poly]]}]

In [20]:
poly

[{'type': 'Polygon',
  'coordinates': [[<shapely.geometry.polygon.Polygon at 0x7f91c3a47c50>]]}]

In [13]:
with fiona.open(aoi, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

In [14]:
shapes

[{'type': 'Polygon',
  'coordinates': [[(-76.46989556858375, 37.25809799243308),
    (-76.35229769099021, 37.25949980504049),
    (-76.35422963761951, 37.36725849755483),
    (-76.4719953783964, 37.365851240046055),
    (-76.46989556858375, 37.25809799243308)]]}]

In [ ]:
test = alltif[0]

In [ ]:
src = rasterio.open(test)

In [ ]:

extent = [src.bounds]
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
print(extent)
ax = rasterio.plot.show(src, extent=extent, ax=ax, cmap='pink', title='My plot')

aio_gdf.plot(ax=ax)
plt.show()

In [ ]:
def Clipper(raster, vector):

    # Read Shapefile
    with fiona.open(vector, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rio.open(raster) as src:
        # read imagery file
        out_image, out_transform = mask.mask(src, shapes, crop=True, nodata=np.nan)

        # Check that after the clip, the image is not empty
        test = out_image[~np.isnan(out_image)]

        if test[test > 0].shape[0] == 0:
            raise RuntimeError("Empty output")

        out_meta = src.profile
        out_meta.update({"height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

    return (out_image, out_meta)


# Clip the raster
array_out, out_meta = Clipper(raster, path)

# Save the clip as a tif
with rio.open(clipped, "w", **out_meta) as dest:
    dest.write(array_out)